In [57]:
import csv
import pandas as pd 
import torch
import torch.nn as nn


In [58]:
from datasets import load_dataset

# s='train[:10]'
# dataset_train = load_dataset('csv', data_files='/kaggle/input/translated-data/Translated_hindi_train.csv')
# dataset_val = load_dataset('csv', data_files='/kaggle/input/summariser-data2022/val.csv')

In [59]:
# dataset_train

In [60]:
df_train = pd.read_csv('/kaggle/input/summariser-data2022/train.csv')     
# df_val = pd.read_csv('/kaggle/input/summariser-data2022/val.csv')     

In [61]:
article_limit = 300
# summary_limit=
def resize(article,summary):
    a=[]
    s=[]
    for i in range(len(article)):
        if(len(article[i].split())>article_limit):
            a.append(article[i])
            s.append(summary[i])
        else:
            continue
    
    return a,s

In [62]:
train_article = df_train['Article'].tolist()[:5000]
train_summary = df_train['Summary'].tolist()[:5000]
train_article,train_summary = resize(train_article,train_summary)

In [63]:
len(train_article)

3097

In [64]:
# from torch.utils.data import Dataset
from datasets import DatasetDict, Dataset

x=100

t=int(len(train_article)*0.9)
print(t)

data_dict = {
    "Article": train_article[:t],
    "Summary": train_summary[:t]
}

# Create a Dataset object from the dictionary
dataset_train = Dataset.from_dict(data_dict)

dict_train = DatasetDict({
    'train': dataset_train,
    # Add more datasets for validation, test, etc., as needed
})

print((dict_train))
# print(t)

2787
DatasetDict({
    train: Dataset({
        features: ['Article', 'Summary'],
        num_rows: 2787
    })
})


In [65]:

# val_article = df_val['Article'].tolist()[:x]
# val_summary = df_val['Summary'].tolist()[:x]

val_article = train_article[t:]
val_summary = train_summary[t:] 

# val_article,val_summary = resize(val_article,val_summary)

data_dict = {
    "Article": val_article,
    "Summary": val_summary
}

# Create a Dataset object from the dictionary
dataset_val = Dataset.from_dict(data_dict)

dict_val = DatasetDict({
    'validation': dataset_val,
    # Add more datasets for validation, test, etc., as needed
})

print(type(dict_val))
dict_val

<class 'datasets.dataset_dict.DatasetDict'>


DatasetDict({
    validation: Dataset({
        features: ['Article', 'Summary'],
        num_rows: 310
    })
})

In [66]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# tokenizer = AutoTokenizer.from_pretrained("ai4bharat/IndicBART")
# model = AutoModelForSeq2SeqLM.from_pretrained("ai4bharat/IndicBART")
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# tokenizer = AutoTokenizer.from_pretrained("Someman/bart-hindi")
# model = AutoModelForSeq2SeqLM.from_pretrained("Someman/bart-hindi")

tokenizer = AutoTokenizer.from_pretrained("csebuetnlp/mT5_multilingual_XLSum")
model = AutoModelForSeq2SeqLM.from_pretrained("csebuetnlp/mT5_multilingual_XLSum")


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


KeyboardInterrupt: 

In [ ]:
# from transformers import AutoTokenizer
# import torch

# df_test = pd.read_csv('/kaggle/input/summariser-data2022/test.csv')
# text= df_test['Article'].tolist()[12]

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = model.to(device)

# inputs = tokenizer(text, return_tensors="pt")

# # Move the input tensors to the same device as the model
# inputs = {k: v.to(device) for k, v in inputs.items()}

# # Generate outputs using the model
# with torch.no_grad():
#     # Access the correct key for `input_ids` in the `inputs` dictionary
#     input_ids = inputs["input_ids"]
#     outputs = model.generate(input_ids, max_new_tokens=70, do_sample=False)

# generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
# print("Generated Text:", generated_text)


In [ ]:
# summary = df_test['Summary'].tolist()[12]
# summary

In [ ]:
# text

In [ ]:
max_input_length = 600
max_target_length = 40

prefix="summarize: "
x=10
def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["Article"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True,padding='max_length')

    # Setup the tokenizer for targets
    labels = tokenizer(text_target=examples["Summary"], max_length=max_target_length, truncation=True,padding='max_length')
#     labels = tokenizer(text_target=examples["Summary"], padding='longest', truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
# inp = preprocess_function(dataset_train['train'])

# x=10
# tokenised_data_train = dataset_train.map(preprocess_function, batched=True)
# tokenised_data_val = dataset_val.map(preprocess_function, batched=True)

tokenised_data_train = dict_train.map(preprocess_function, batched=True)
tokenised_data_val = dict_val.map(preprocess_function, batched=True)

In [ ]:
(tokenised_data_train)

In [ ]:
import nltk
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    # Note that other metrics may not have a `use_aggregator` parameter
    # and thus will return a list, computing a metric for each sentence.
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True, use_aggregator=True)
    # Extract a few results
    result = {key: value * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model='mT5_multilingual_XLSum')

In [ ]:
!pip install rouge_score
!pip install evaluate

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
import evaluate

rouge = evaluate.load("rouge")


batch_size = 1
# model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    output_dir="mT5_multilingual_XLSum",
#     eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=1,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
tokenised_data_train

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenised_data_train["train"],
    eval_dataset=tokenised_data_val["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
#      data_collator=data_collator,
)

trainer.train()

In [ ]:
print()

In [ ]:
text = df_train['Article'].tolist()[11]
text='summarise: '+text
text

In [ ]:
# Save tokenizer
import pickle
with open('/kaggle/working/tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)

In [ ]:
import pickle

# Save model
with open('mt5_xl_23_translated_model.pkl', 'wb') as f:
    pickle.dump(model, f)

# Save tokenizer
with open('mt5_xl_23_translated_tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)


In [ ]:
import pandas as pd

df_test = pd.read_csv('/kaggle/input/summariser-data2022/test.csv')

In [ ]:
from transformers import pipeline
# prin
with open('/kaggle/working/Indic_Bart.pkl', 'rb') as f:
    model = pickle.load(f)
text= df_test['Article'].tolist()[12]
summarizer = pipeline("summarization", model=model,tokenizer=AutoTokenizer.from_pretrained("/kaggle/working/tokeniser"))
summarizer(text)

In [ ]:
text

In [ ]:
t=df_test['Summary'].tolist()[12]
t

In [ ]:
with open('/kaggle/working/Indic_Bart.pkl', 'rb') as f:
    model = pickle.load(f)
tokenizer = AutoTokenizer.from_pretrained("/kaggle/working/tokeniser")

In [ ]:
from transformers import AutoTokenizer
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

inputs = tokenizer(text, return_tensors="pt")

# Move the input tensors to the same device as the model
inputs = {k: v.to(device) for k, v in inputs.items()}

# Generate outputs using the model
with torch.no_grad():
    # Access the correct key for `input_ids` in the `inputs` dictionary
    input_ids = inputs["input_ids"]
    outputs = model.generate(input_ids, max_new_tokens=100, do_sample=False)

generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Generated Text:", generated_text)


In [ ]:
text

In [ ]:
t=df_test['Summary'].tolist()[12]
t

In [ ]:
text

In [ ]:
!pip install indic-nlp-library


In [ ]:
from transformers import AutoTokenizer
import torch

# with open('/kaggle/working/Indic_Bart.pkl', 'rb') as f:
#     model = pickle.load(f)
# tokenizer = AutoTokenizer.from_pretrained("/kaggle/working/tokeniser")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# def summarise(s):

inputs = tokenizer(text, return_tensors="pt")

# Move the input tensors to the same device as the model
inputs = {k: v.to(device) for k, v in inputs.items()}

# Generate outputs using the model
with torch.no_grad():
    # Access the correct key for `input_ids` in the `inputs` dictionary
    input_ids = inputs["input_ids"]
    outputs = model.generate(input_ids, max_new_tokens=80, do_sample=False)

generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
# return generated_text
print("Generated Text:", generated_text)

In [ ]:
summaries = []
ori_summary = []
s=df_test['Summary'].tolist()
a = df_test['Article'].tolist()
print(len(s))
ori_summary.append(s[0])
print(a[0])
# summaries.append(summarise(a[0]))
# for i in range(len(s)):
#     ori_summary.append(s[i])
#     summaries.append(summarise(a[i]))

In [ ]:
!pip install rouge

In [ ]:
df_test2=df_train[:100]
df_test2

In [ ]:

def predict_summary(text):
    inputs = tokenizer(text, return_tensors='pt', max_length=600, truncation=True)
    inputs.to("cuda")
    summary_ids = model.generate(inputs['input_ids'], num_beams=4, max_length=40, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

df_test2['predicted_summary'] = df_test2['Article'].apply(predict_summary)

In [ ]:
from rouge import Rouge

rouge = Rouge()


def calculate_rouge(row):
    return rouge.get_scores(row['predicted_summary'], row['Summary'], avg=True)

df_test2['rouge'] = df_test2.apply(calculate_rouge, axis=1)
df_test2['rouge-1'] = df_test2['rouge'].apply(lambda x: x['rouge-1']['f'])
df_test2['rouge-2'] = df_test2['rouge'].apply(lambda x: x['rouge-2']['f'])

print(df_test2['rouge-1'].mean())
print(df_test2['rouge-2'].mean())

In [ ]:
!pip install rouge

In [ ]:
# # Define compute_metrics function
# def compute_metrics(eval_pred):
#     predictions, labels = eval_pred
#     decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
#     labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
#     decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

#     # Load ROUGE and BERT score metrics
#     rouge = load_metric('rouge')
#     bert_score = load_metric('bertscore')

#     # Compute ROUGE and BERT scores
#     rouge_output = rouge.compute(predictions=decoded_preds, references=decoded_labels, rouge_types=["rougeL", "rouge2"])
#     bert_score_output = bert_score.compute(predictions=decoded_preds, references=decoded_labels, lang="hi")

#     return {
#         "rouge2": rouge_output['rouge2'].mid.fmeasure,
#         "rougeL": rouge_output['rougeL'].mid.fmeasure,
#         "bert_score": bert_score_output['f1'][0]
#     }

********************************************

**Testing Phase**

***********************************************

In [1]:
!pip install rouge_score
!pip install rouge

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=32f8930671282f37d01c18b5d5c137b46ee9d17c5fbbb74c73c26d10db9779df
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [2]:
import pickle
import torch
import pandas as pd


with open('/kaggle/working/mt5_xl_23_translated_model.pkl', 'rb') as f:
    model = pickle.load(f)
    
with open('/kaggle/working/mt5_xl_23_translated_tokenizer.pkl', 'rb') as f:
    tokenizer = pickle.load(f)
    

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
# model.to(tokenizer)

def file_read(path):
    return pd.read_csv(path)

df_val = file_read('/kaggle/input/summariser-data2022/train.csv')
df_test = file_read('/kaggle/input/summariser-data2022/test.csv')

df_test = df_test.sample(frac=0.2)



from rouge import Rouge
def predict_summary(text):
    inputs = tokenizer(text, return_tensors='pt', max_length=600, truncation=True)
    inputs.to("cuda")
    summary_ids = model.generate(inputs['input_ids'], num_beams=4, max_length=40, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

def score(df):

    df['predicted_summary'] = df['Article'].apply(predict_summary)

    rouge = Rouge()


    def calculate_rouge(row):
        return rouge.get_scores(row['predicted_summary'], row['Summary'], avg=True)

    df['rouge'] = df_test.apply(calculate_rouge, axis=1)
    df['rouge-1'] = df_test['rouge'].apply(lambda x: x['rouge-1']['f'])
    df['rouge-2'] = df_test['rouge'].apply(lambda x: x['rouge-2']['f'])

    print(df['rouge-1'].mean())
    print(df['rouge-2'].mean())

    df_test['rouge-l'] = df_test['rouge'].apply(lambda x: x['rouge-l']['f'])
    print(df_test['rouge-l'].mean())

score(df_test)

score(df_val)

print("Rouge-1 score :" ,df_test['rouge-1'].mean())
print("Rouge-2 score :", df_test['rouge-2'].mean())
print("Rouge-L score :", df_test['rouge-l'].mean())

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/working/mt5_xl_23_translated_model.pkl'